In [1]:
import seaborn as sns
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler, Normalizer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import janitor
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df.head()

,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,...,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
0,806407308,Attrited Customer,54,F,3,College,Single,Unknown,Blue,49,...,2,3,13184.0,0,13184.0,1.166,2047,33,0.179,0.000
1,801786483,Existing Customer,51,M,0,High School,Single,$80K - $120K,Blue,45,...,2,3,10648.0,2517,8131.0,0.708,4930,89,0.978,0.236
2,779870883,Existing Customer,45,F,4,High School,Single,$40K - $60K,Blue,29,...,2,1,4595.0,1781,2814.0,0.708,3746,68,0.545,0.388
3,720336708,Existing Customer,53,M,3,Doctorate,Married,$40K - $60K,Blue,35,...,3,2,3789.0,1706,2083.0,1.047,1609,42,0.680,0.450
4,716769783,Existing Customer,48,F,2,Graduate,Married,Unknown,Blue,40,...,2,4,5104.0,736,4368.0,0.679,3943,78,0.696,0.144


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   clientnum                 8101 non-null   int64  
 1   attrition_flag            8101 non-null   object 
 2   customer_age              8101 non-null   int64  
 3   gender                    8101 non-null   object 
 4   dependent_count           8101 non-null   int64  
 5   education_level           8101 non-null   object 
 6   marital_status            8101 non-null   object 
 7   income_category           8101 non-null   object 
 8   card_category             8101 non-null   object 
 9   months_on_book            8101 non-null   int64  
 10  total_relationship_count  8101 non-null   int64  
 11  months_inactive_12_mon    8101 non-null   int64  
 12  contacts_count_12_mon     8101 non-null   int64  
 13  credit_limit              8101 non-null   float64
 14  total_re

In [5]:
one_hot = OneHotEncoder()
min_max = MinMaxScaler()
sd = StandardScaler()
le = LabelEncoder()

### Individual Feature Engineering

In [6]:
ct = ColumnTransformer([("ohe", one_hot, ["marital_status"])])

In [7]:
x = ct.fit_transform(df).toarray()

In [8]:
df_new = df.copy()

In [9]:
element = sorted(df["marital_status"].unique())
element[0].lower()
for i in range(len(element)):
    df_new[element[i].lower()] = x[:,i]
df_new = df_new.drop("marital_status", axis = 1)

In [10]:
df_new.head()

,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,income_category,card_category,months_on_book,total_relationship_count,...,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,divorced,married,single,unknown
0,806407308,Attrited Customer,54,F,3,College,Unknown,Blue,49,6,...,13184.0,1.166,2047,33,0.179,0.000,0.0,0.0,1.0,0.0
1,801786483,Existing Customer,51,M,0,High School,$80K - $120K,Blue,45,3,...,8131.0,0.708,4930,89,0.978,0.236,0.0,0.0,1.0,0.0
2,779870883,Existing Customer,45,F,4,High School,$40K - $60K,Blue,29,3,...,2814.0,0.708,3746,68,0.545,0.388,0.0,0.0,1.0,0.0
3,720336708,Existing Customer,53,M,3,Doctorate,$40K - $60K,Blue,35,5,...,2083.0,1.047,1609,42,0.680,0.450,0.0,1.0,0.0,0.0
4,716769783,Existing Customer,48,F,2,Graduate,Unknown,Blue,40,3,...,4368.0,0.679,3943,78,0.696,0.144,0.0,1.0,0.0,0.0


In [11]:
## One Hot Encoding Function
def one_hot_func(df, var):
    df_use = df.copy()
    element = sorted(df_use[var].unique())
    element[0].lower()
    ct = ColumnTransformer([("ohe", one_hot, [var])])
    x = ct.fit_transform(df).toarray()
    for i in range(len(element)):
        df_use[var + "_" + element[i].lower()] = x[:,i]
    df_use = df_use.drop(var, axis = 1)
    return df_use

In [12]:
df_update = one_hot_func(df, "marital_status")

In [13]:
df_update.head()

,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,income_category,card_category,months_on_book,total_relationship_count,...,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,marital_status_divorced,marital_status_married,marital_status_single,marital_status_unknown
0,806407308,Attrited Customer,54,F,3,College,Unknown,Blue,49,6,...,13184.0,1.166,2047,33,0.179,0.000,0.0,0.0,1.0,0.0
1,801786483,Existing Customer,51,M,0,High School,$80K - $120K,Blue,45,3,...,8131.0,0.708,4930,89,0.978,0.236,0.0,0.0,1.0,0.0
2,779870883,Existing Customer,45,F,4,High School,$40K - $60K,Blue,29,3,...,2814.0,0.708,3746,68,0.545,0.388,0.0,0.0,1.0,0.0
3,720336708,Existing Customer,53,M,3,Doctorate,$40K - $60K,Blue,35,5,...,2083.0,1.047,1609,42,0.680,0.450,0.0,1.0,0.0,0.0
4,716769783,Existing Customer,48,F,2,Graduate,Unknown,Blue,40,3,...,4368.0,0.679,3943,78,0.696,0.144,0.0,1.0,0.0,0.0


### Multiple Feature Engineering

In [14]:
ct = ColumnTransformer([("ohe", one_hot, ["marital_status","gender","income_category"]),("min_max",min_max,["total_revolving_bal","credit_limit"])])

In [15]:
new_arr = ct.fit_transform(df)

In [16]:
new_arr

array([[0.        , 0.        , 1.        , ..., 1.        , 0.        ,
        0.35509422],
       [0.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.27842625],
       [0.        , 0.        , 1.        , ..., 0.        , 0.7075884 ,
        0.09543287],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 0.19825189,
        0.08361222],
       [0.        , 1.        , 0.        , ..., 0.        , 0.53277712,
        0.18056576],
       [0.        , 0.        , 1.        , ..., 0.        , 0.40405244,
        0.34393867]])

In [17]:
new_arr.shape

(8101, 14)

In [18]:
import numpy as np

In [19]:
df["gender"].unique()

array(['F', 'M'], dtype=object)

In [20]:
#columns = ["marital_status", "gender", "income_category", "total_revolving_bal","credit_limit"]
columns = ["marital_status", "gender", "income_category", "total_revolving_bal","credit_limit"]

new_df = pd.DataFrame(index = range(df.shape[0]))
for col in columns:
    if is_numeric_dtype(df[col]):
        ct = ColumnTransformer([("min_max", min_max, [col])])
        new_df[col + "_scaled"] = ct.fit_transform(df)
        #df = df.drop(col, axis = 1)
    elif is_string_dtype(df[col]):
        elements = sorted(df[col].unique())
        ct2 = ColumnTransformer([("ohe", one_hot, [col])])
        if len(elements) > 2:
            x = ct2.fit_transform(df).toarray()
        elif len(elements) <= 2:
            x = np.array(ct2.fit_transform(df))
        for i in range(len(elements)):
            new_df[col + "_" + elements[i].lower()] = x[:,i]
        #df = df.drop(col, axis = 1)

In [21]:
new_df

,marital_status_divorced,marital_status_married,marital_status_single,marital_status_unknown,gender_f,gender_m,income_category_$120k +,income_category_$40k - $60k,income_category_$60k - $80k,income_category_$80k - $120k,income_category_less than $40k,income_category_unknown,total_revolving_bal_scaled,credit_limit_scaled
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.355094
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.278426
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.707588,0.095433
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.677791,0.071066
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.292412,0.110821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.394120,0.040592
8097,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.540723,0.077717
8098,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.198252,0.083612
8099,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.532777,0.180566


### Feature Engineering on all features

In [22]:
df_use = df.drop(["clientnum", "attrition_flag"], axis = 1)
df_use.head()

,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
0,54,F,3,College,Single,Unknown,Blue,49,6,2,3,13184.0,0,13184.0,1.166,2047,33,0.179,0.000
1,51,M,0,High School,Single,$80K - $120K,Blue,45,3,2,3,10648.0,2517,8131.0,0.708,4930,89,0.978,0.236
2,45,F,4,High School,Single,$40K - $60K,Blue,29,3,2,1,4595.0,1781,2814.0,0.708,3746,68,0.545,0.388
3,53,M,3,Doctorate,Married,$40K - $60K,Blue,35,5,3,2,3789.0,1706,2083.0,1.047,1609,42,0.680,0.450
4,48,F,2,Graduate,Married,Unknown,Blue,40,3,2,4,5104.0,736,4368.0,0.679,3943,78,0.696,0.144


In [23]:
num = []
cat = []
columns = list(df_use.columns)
for col in columns:
    if is_numeric_dtype(df_use[col]):
        num.append(col)
    elif is_string_dtype(df[col]):
        cat.append(col)

In [24]:
num

['customer_age',
 'dependent_count',
 'months_on_book',
 'total_relationship_count',
 'months_inactive_12_mon',
 'contacts_count_12_mon',
 'credit_limit',
 'total_revolving_bal',
 'avg_open_to_buy',
 'total_amt_chng_q4_q1',
 'total_trans_amt',
 'total_trans_ct',
 'total_ct_chng_q4_q1',
 'avg_utilization_ratio']

In [25]:
cat

['gender',
 'education_level',
 'marital_status',
 'income_category',
 'card_category']

In [26]:
ct3 = ColumnTransformer([("min_max", min_max, num), ("one_hot", one_hot, cat)])

In [27]:
df_arr = ct3.fit_transform(df)

In [28]:
df_arr.shape

(8101, 37)

In [29]:
df_arr

array([[0.59574468, 0.6       , 0.8372093 , ..., 0.        , 0.        ,
        0.        ],
       [0.53191489, 0.        , 0.74418605, ..., 0.        , 0.        ,
        0.        ],
       [0.40425532, 0.8       , 0.37209302, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.5106383 , 0.2       , 0.53488372, ..., 0.        , 0.        ,
        0.        ],
       [0.36170213, 1.        , 0.53488372, ..., 0.        , 0.        ,
        0.        ],
       [0.59574468, 0.4       , 0.53488372, ..., 0.        , 0.        ,
        0.        ]])

In [30]:
def column_transform(df, columns):
    new_df = df.copy()
    for col in columns:
        if is_numeric_dtype(df[col]):
            ct = ColumnTransformer([("min_max", min_max, [col])])
            new_df[col + "_scaled"] = ct.fit_transform(df)
        elif is_string_dtype(df[col]):
            elements = sorted(df[col].unique())
            ct2 = ColumnTransformer([("ohe", one_hot, [col])])
            if len(elements) > 2:
                x = ct2.fit_transform(df).toarray()
            elif len(elements) <= 2:
                x = np.array(ct2.fit_transform(df))
            for i in range(len(elements)):
                new_df[col + "_" + elements[i].lower()] = x[:,i]
    new_df = new_df.drop(columns, axis = 1)
    return new_df

In [31]:
df_renewed = column_transform(df_use, list(df_use.columns))

In [32]:
df_renewed["attrition_flag"] = df["attrition_flag"].replace(["Attrited Customer","Existing Customer"], [1,0]).astype(int)
df_renewed["clientnum"] = df["clientnum"]

In [33]:
df_renewed = df_renewed.clean_names()

In [34]:
df_renewed.head()

,customer_age_scaled,gender_f,gender_m,dependent_count_scaled,education_level_college,education_level_doctorate,education_level_graduate,education_level_high_school,education_level_post_graduate,education_level_uneducated,...,credit_limit_scaled,total_revolving_bal_scaled,avg_open_to_buy_scaled,total_amt_chng_q4_q1_scaled,total_trans_amt_scaled,total_trans_ct_scaled,total_ct_chng_q4_q1_scaled,avg_utilization_ratio_scaled,attrition_flag,clientnum
0,0.595745,1.0,0.0,0.6,1.0,0.0,0.0,0.0,0.0,0.0,...,0.355094,0.000000,0.381914,0.343244,0.084494,0.178295,0.050126,0.000000,1,806407308
1,0.531915,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.278426,1.000000,0.235505,0.208419,0.245071,0.612403,0.273873,0.236236,0,801786483
2,0.404255,1.0,0.0,0.8,0.0,0.0,0.0,1.0,0.0,0.0,...,0.095433,0.707588,0.081448,0.208419,0.179124,0.449612,0.152618,0.388388,0,779870883
3,0.574468,0.0,1.0,0.6,0.0,1.0,0.0,0.0,0.0,0.0,...,0.071066,0.677791,0.060267,0.308213,0.060098,0.248062,0.190423,0.450450,0,720336708
4,0.468085,1.0,0.0,0.4,0.0,0.0,1.0,0.0,0.0,0.0,...,0.110821,0.292412,0.126474,0.199882,0.190097,0.527132,0.194903,0.144144,0,716769783


In [35]:
df_renewed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8101 entries, 0 to 8100
Data columns (total 39 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   customer_age_scaled              8101 non-null   float64
 1   gender_f                         8101 non-null   float64
 2   gender_m                         8101 non-null   float64
 3   dependent_count_scaled           8101 non-null   float64
 4   education_level_college          8101 non-null   float64
 5   education_level_doctorate        8101 non-null   float64
 6   education_level_graduate         8101 non-null   float64
 7   education_level_high_school      8101 non-null   float64
 8   education_level_post_graduate    8101 non-null   float64
 9   education_level_uneducated       8101 non-null   float64
 10  education_level_unknown          8101 non-null   float64
 11  marital_status_divorced          8101 non-null   float64
 12  marital_status_marri

In [36]:
df_renewed.to_csv("data/train_fe.csv")